In [1]:
# Importamos los módulos.
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
# Definimos los archivos árbol y raíz.
tree = ET.parse('export.xml')
root = tree.getroot()

In [3]:
# Creamos las columnas que utilizaremos para el dataframe.
# bpm = frecuencia cardiaca, time = hora de la medida, set = conjunto de la medida, creationDate = fecha de creación del set
col = ['bpm', 'time', 'set', 'creationDate']
rows = []

In [4]:
set_ = 0
for i in root.findall("Record/[@type='HKQuantityTypeIdentifierHeartRateVariabilitySDNN']"):
    # Recorremos los registros de las medidas de la variabilidad de la frecuencia cardiaca en el archivo bruto
    for j in i:
        # Extraemos la fecha de cada conjunto de datos (set)
        date_ = i.attrib.get('creationDate')
        for k in j:
            # Extraemos los valores de bpm y time y creamos el dataset
            bpm_ = k.attrib.get('bpm')
            time_ = k.attrib.get('time')
            rows.append({'bpm': bpm_, 'time': time_, 'set':set_, 'creationDate': date_})
        set_ = set_ + 1

In [5]:
# Creamos el dataframe a partir del dataset
df = pd.DataFrame(rows, columns = col)

In [6]:
df

,bpm,time,set,creationDate
0,92,"19:02:42,97",0,2021-04-06 19:07:44 +0200
1,94,"19:02:43,61",0,2021-04-06 19:07:44 +0200
2,96,"19:02:44,23",0,2021-04-06 19:07:44 +0200
3,93,"19:02:44,88",0,2021-04-06 19:07:44 +0200
4,92,"19:02:45,53",0,2021-04-06 19:07:44 +0200
...,...,...,...,...
29984,84,"17:11:13,27",141,2021-05-12 17:11:51 +0200
29985,90,"17:11:13,94",141,2021-05-12 17:11:51 +0200
29986,86,"17:11:14,64",141,2021-05-12 17:11:51 +0200
29987,80,"17:11:38,62",141,2021-05-12 17:11:51 +0200


In [7]:
# Creamos la columna measureDate con la fecha de la medida
sep_date = df['creationDate'].str.split(expand=True)
sep_date.columns = ['date', 'creationTime', 'timezone']
df = pd.concat([df, sep_date], axis=1)
df['measureDate'] = df[['date', 'time']].apply(' '.join, axis=1)

In [8]:
# Eliminamos las columnas que no nos interesan
df.drop(columns=['date','creationTime','timezone'],inplace=True)

In [9]:
df

,bpm,time,set,creationDate,measureDate
0,92,"19:02:42,97",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:42,97"
1,94,"19:02:43,61",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:43,61"
2,96,"19:02:44,23",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:44,23"
3,93,"19:02:44,88",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:44,88"
4,92,"19:02:45,53",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:45,53"
...,...,...,...,...,...
29984,84,"17:11:13,27",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:13,27"
29985,90,"17:11:13,94",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:13,94"
29986,86,"17:11:14,64",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:14,64"
29987,80,"17:11:38,62",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:38,62"


In [10]:
# Exportamos el dataframe a un archivo csv
df.to_csv("HRV_Extractor.csv", index=False)